# Transcript from Lecture 4, January 24, 2023


In [1]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2023-winter/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# Unit lower triangular solve

In [2]:
L = np.array([[1,0,0,0], [1,1,0,0], [2,0,1,0], [-1,2,1,1]])
print("L:\n", L)

L:
 [[ 1  0  0  0]
 [ 1  1  0  0]
 [ 2  0  1  0]
 [-1  2  1  1]]


In [3]:
b = np.array([2,3,3,1])
print("b:", b)

b: [2 3 3 1]


In [4]:
y = npla.solve(L, b)
print("y:", y)

y: [ 2.  1. -1.  2.]


In [5]:
cs111.Lsolve(L, b)

array([ 2.,  1., -1.,  2.])

In [6]:
print("L @ y:", L @ y)

L @ y: [2. 3. 3. 1.]


<b>Let L be an n-by-n unit lower triangular matrix, and b an n-vector.<br><br>
What's the asymptotic complexity of x = cs111.Lsolve(L, b), as O(...)?

# LU factorization of a matrix

<b>Theorem: If square matrix *A* is not singular, then there exist a permutation matrix *P*, a unit lower 
    triangular matrix *L*, and an upper triangular matrix *U* (not necessarily unit upper triangular)
    such that
    
    P @ A == L @ U
    
<b>or equivalently
    
    A = P.T @ L @ U

In [7]:
A = np.array(
      [[ 0.  ,  4.  ,  4.  ,  1.  ],
       [ 2.  , -1.5 , -1.25,  2.25],
       [ 4.  ,  1.  ,  1.5 ,  1.5 ],
       [ 8.  ,  2.  ,  1.  ,  1.  ]])

print('A:\n', A)

A:
 [[ 0.    4.    4.    1.  ]
 [ 2.   -1.5  -1.25  2.25]
 [ 4.    1.    1.5   1.5 ]
 [ 8.    2.    1.    1.  ]]


In [8]:
# Our cs111.LUfactor() returns the permutation as an array p, not a matrix P

L,U,p = cs111.LUfactor(A)

In [9]:
print('L:\n', L)
print('\nU:\n', U)
print('\np:', p)

L:
 [[ 1.    0.    0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.5   0.    1.    0.  ]
 [ 0.25 -0.5   0.5   1.  ]]

U:
 [[8. 2. 1. 1.]
 [0. 4. 4. 1.]
 [0. 0. 1. 1.]
 [0. 0. 0. 2.]]

p: [3 0 2 1]


In [10]:
print('L @ U\n', L@U)

L @ U
 [[ 8.    2.    1.    1.  ]
 [ 0.    4.    4.    1.  ]
 [ 4.    1.    1.5   1.5 ]
 [ 2.   -1.5  -1.25  2.25]]


In [11]:
print('A[p,:]\n', A[p,:])

A[p,:]
 [[ 8.    2.    1.    1.  ]
 [ 0.    4.    4.    1.  ]
 [ 4.    1.    1.5   1.5 ]
 [ 2.   -1.5  -1.25  2.25]]


# Solving Ax = b by using the LU factorization

In [12]:
# Get a right-hand side

b = np.random.random(4)
print('b:', b)

b: [0.0317 0.5563 0.6567 0.9838]


In [13]:
# Here's the answer numpy gives us

x = npla.solve(A,b)
x

array([ 0.1187, -0.0654,  0.0429,  0.1219])

In [14]:
p

array([3, 0, 2, 1])

In [ ]:
# Here's the answer we get by using the matrix factorization

In [15]:
print('b[p]:', b[p])

b[p]: [0.9838 0.0317 0.6567 0.5563]


In [16]:
y = cs111.Lsolve(L, b[p])
print('y:', y)

y: [0.9838 0.0317 0.1648 0.2438]


In [17]:
x = cs111.Usolve(U, y)
print('x:', x)

x: [ 0.1187 -0.0654  0.0429  0.1219]


In [18]:
# Check the answer

print('b =', b)
print()

print("Ax =", A @ x)
print()

print("Residual: b - Ax =", b - A @ x)
print()

print("Residual norm: ||(b - Ax)|| =", npla.norm(b - A@x))
print()

print("Relative residual norm: ||(b - Ax)|| / ||b|| =", npla.norm(b-A@x)/npla.norm(b))

b = [0.0317 0.5563 0.6567 0.9838]

Ax = [0.0317 0.5563 0.6567 0.9838]

Residual: b - Ax = [0.0000e+00 1.1102e-16 0.0000e+00 0.0000e+00]

Residual norm: ||(b - Ax)|| = 1.1102230246251565e-16

Relative residual norm: ||(b - Ax)|| / ||b|| = 8.49095972596132e-17


Where do all those tiny numbers come from? Floating-point! We'll get back to that later.

# cs111.LUsolve() packages this all up in one call

In [19]:
x, rel_res = cs111.LUsolve(A,b)

print('x:', x)
print()
print('relative residual:', rel_res)

x: [ 0.1187 -0.0654  0.0429  0.1219]

relative residual: 8.49095972596132e-17


# Factoring A = LU by Gaussian elimination

In [20]:
# Let's get a random matrix for an example

n = 5

A = np.round(np.random.random((n,n))*100)
A

array([[12., 60., 99., 69., 87.],
       [74., 37., 33., 78., 94.],
       [23., 39., 41., 82., 79.],
       [19., 39.,  3., 24., 50.],
       [54., 74., 36., 93., 36.]])

In [21]:
b = np.random.random(n)
print("b:", b)

b: [0.8194 0.6234 0.8539 0.663  0.335 ]


In [22]:
L, U = cs111.LUfactorNoPiv(A)

In [23]:
L

array([[1.    , 0.    , 0.    , 0.    , 0.    ],
       [6.1667, 1.    , 0.    , 0.    , 0.    ],
       [1.9167, 0.2282, 1.    , 0.    , 0.    ],
       [1.5833, 0.1682, 3.3415, 1.    , 0.    ],
       [4.5   , 0.5886, 4.106 , 1.0675, 1.    ]])

In [24]:
U

array([[  12.    ,   60.    ,   99.    ,   69.    ,   87.    ],
       [   0.    , -333.    , -577.5   , -347.5   , -442.5   ],
       [   0.    ,    0.    ,  -16.9482,   29.0593,   13.241 ],
       [   0.    ,    0.    ,    0.    , -123.9141,  -57.5807],
       [   0.    ,    0.    ,    0.    ,    0.    ,  -87.9474]])

In [25]:
print("b:", b)
print()

y = cs111.Lsolve(L, b)
print("y:", y)
print()

x = cs111.Usolve(U, y)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.8194 0.6234 0.8539 0.663  0.335 ]

y: [ 0.8194 -4.4296  0.2943 -0.8729 -1.0216]

x: [-0.0084  0.0056 -0.0055  0.0016  0.0116]

A @ x: [0.8194 0.6234 0.8539 0.663  0.335 ]

b - A @ x: [ 0.0000e+00 -6.6613e-16 -1.1102e-16  0.0000e+00  1.6653e-16]


<b>What is the asymptotic running time of cs111.LUfactorNoPiv(A)?

# Permuting rows of A : Partial pivoting

In [26]:
A = np.array([[1,1,2], [1,1,3], [2,3,4]])
A

array([[1, 1, 2],
       [1, 1, 3],
       [2, 3, 4]])

In [27]:
L, U = cs111.LUfactorNoPiv(A)

AssertionError: pivot is zero, can't continue

In [28]:
# LU factorization with partial pivoting ('partial pivoting' means 'permute rows, not columns, of A')
L, U, p = cs111.LUfactor(A)

In [29]:
L

array([[1. , 0. , 0. ],
       [0.5, 1. , 0. ],
       [0.5, 1. , 1. ]])

In [30]:
U

array([[ 2. ,  3. ,  4. ],
       [ 0. , -0.5,  1. ],
       [ 0. ,  0. , -1. ]])

In [31]:
# The permutation of the rows of A
print("p:", p)

p: [2 1 0]


In [32]:
L @ U

array([[2., 3., 4.],
       [1., 1., 3.],
       [1., 1., 2.]])

In [33]:
A

array([[1, 1, 2],
       [1, 1, 3],
       [2, 3, 4]])

In [34]:
A[p, :]

array([[2, 3, 4],
       [1, 1, 3],
       [1, 1, 2]])

In [35]:
n = A.shape[0]
b = np.random.random(n)
print("b:", b)

b: [0.5646 0.2334 0.7285]


In [36]:
# A complete solve with partial pivoting
print("b:", b)
print()

print("b[p]:", b[p])
print()

y = cs111.Lsolve(L, b[p])
print("y:", y)
print()

x = cs111.Usolve(U, y)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.5646 0.2334 0.7285]

b[p]: [0.7285 0.2334 0.5646]

y: [ 0.7285 -0.1309  0.3313]

x: [ 1.6279 -0.4008 -0.3313]

A @ x: [0.5646 0.2334 0.7285]

b - A @ x: [-1.1102e-16 -1.1102e-16 -1.1102e-16]


<b>What is the asymptotic running time of L, U, p = cs111.LUfactor(A)?

# Putting it all together: LUsolve

In [37]:
print("b:", b)
print()

x, rel_res = cs111.LUsolve(A, b)
print("x:", x)
print()

print("A @ x:", A @ x)
print()

print("b - A @ x:", b - A @ x)

b: [0.5646 0.2334 0.7285]

x: [ 1.6279 -0.4008 -0.3313]

A @ x: [0.5646 0.2334 0.7285]

b - A @ x: [-1.1102e-16 -1.1102e-16 -1.1102e-16]


<b>What is the asymptotic running time of L, U, p = cs111.LUsolve(A)?

# How good is the answer? Measuring accuracy

In [ ]:
# Next time!